In [ ]:
import yfinance as yf
import pandas as pd

# Define the S&P 500 ticker and the time period
sp500_ticker = '^GSPC'
start_date = '2023-01-01'
end_date = today

# Get the list of S&P 500 tickers
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

In [ ]:
# Download historical data
stock_data = yf.download(sp500_tickers, start=start_date, end=end_date)['Adj Close']
index_data = yf.download(sp500_ticker, start=start_date, end=end_date)['Adj Close']




In [ ]:
# Function to calculate the 200-day moving average
def calculate_200_day_moving_average(stock_prices):
    return stock_prices.rolling(window=200).mean()

In [ ]:
# Function to calculate relative strength
def calculate_relative_strength(stock_prices, index_prices):
    #computes the percentage change between consecutive days for the stock prices.
    #dropna() is used to remove any NaN values that might result from the percentage change calculation (the first value is NaN
    stock_returns = stock_prices.pct_change().dropna()
    index_returns = index_prices.pct_change().dropna()
    #(1 + stock_returns) converts daily returns into a factor by adding 1 (e.g., a return of 2% becomes 1.02)
    #cumprod() computes the cumulative product of the daily returns, resulting in the cumulative return of the stock.
    relative_strength = (1 + stock_returns).cumprod() / (1 + index_returns).cumprod()
    return relative_strength.iloc[-1]

In [ ]:
#Selecting only the values that should be over the 200 days line

filtered_tickers = []
for ticker in sp_500_tickers:
    try:
        current_price = stock_data[ticker].iloc[-1]
        ma_200 = calculate_200_day_moving_average(stock_data[ticker]).iloc[-1]
        if current_price > ma_200:
            filtered_tickers.append(ticker)
    except Exception as e:
        print(f"Could not calculate 200-day MA for {ticker}: {e}")

In [ ]:
# Calculate relative strength for each stock
rs_values = {}
for ticker in filtered_tickers:
    try:
        #function of calculate_relative_strength() is called for each stock in the list of S&P 500 ticker
        rs = calculate_relative_strength(stock_data[ticker], index_data)
        rs_values[ticker] = rs
    except Exception as e:
        print(f"Could not calculate RS for {ticker}: {e}")

In [ ]:
# Convert to DataFrame and sort by RS
rs_df = pd.DataFrame(list(rs_values.items()), columns=['Ticker', 'Relative Strength'])
rs_df = rs_df.sort_values(by='Relative Strength', ascending=False)

# Display the top 10 stocks by relative strength
print(rs_df.head(10))